In [12]:
for key in USABLE_FRAMES.keys():
    for base in MATCHING_RESULTS.keys():
        SS = {
            "X": 0,
            "Z": 0
        }
        for i in range(len(key)):
            if base[i] == "X":
                bit = int(key[i].split(",")[0][0])
            else:
                bit = int(key[i].split(",")[1][0])
                
            SS[base[i]] ^= bit

        ss = ''.join(map(str, SS.values()))
        
        if base == ("X", "X", "X") or base == ("Z", "Z", "Z") or base == ("X", "Z", "X") or base == ("Z", "X", "Z"):
            ss += "0"
        else:
            ss += "1"

        MATCHING_RESULTS_DERIVATION[USABLE_FRAMES[key]][ss] = MATCHING_RESULTS[base]

In [1]:
from alice import Alice
from bob import Bob

def execute_qkd(pairs, DEBUG = True, depolarize_probability = 0):
    # Initialization
    alice = Alice(pairs)
    bob = Bob(depolarize_probability)
    
    # Send pairs
    pairs = alice.prepare()

    # Measure each pair and retrieve double matchings
    double_matchings = bob.measure(pairs)
    
    # Verify that there's double matchings
    if not len(double_matchings):
        print("Key exchange failed, there's no double matchings. Please, try again.")
        return (), ()
    
    # Compute usable frames for Bob
    usable_frames, usable_frames_types = alice.compute_usable_frames(double_matchings)
    
    # Veirfy that there's usable frames
    if not len(usable_frames):
        print("Key exchange failed, there's no usable frames. Please, try again.")
        return (), ()

    # Compute sifting string
    sifting_string = bob.compute_sifting_string(usable_frames)

    # Compute measured string
    measured_string = bob.compute_measured_string(usable_frames)

    # Bob computes secret key
    bob_key = bob.generate_shared_key(usable_frames)

    # Alice computes secret key
    alice_key = alice.generate_shared_key(usable_frames_types, sifting_string)
    
    if alice_key == bob_key:
        if DEBUG:
            print("Key exchange completed, here's the data:")
            print(f"Alice send pairs: {alice.pairs_data}")
            print(f"Bob send double matchings: {double_matchings}")
            print(f"Alice send usable frames: {usable_frames}")
            print(f"Bob send sifting string: {sifting_string}")
            print(f"Bob shared key: {bob_key}")
            print(f"Alice shared key: {alice_key}")
        return (usable_frames, sifting_string, measured_string), (alice_key, bob_key)
    else:
        print("Key exchange failed, both keys are not equal. Please, try again.")
        return (usable_frames, sifting_string, measured_string), (alice_key, bob_key)

In [2]:
public, private = execute_qkd(16)

Key exchange completed, here's the data:
Alice send pairs: {0: '0x,1z', 1: '0x,0z', 2: '0x,1z', 3: '0x,0z', 4: '0x,1z', 5: '0x,1z', 6: '1x,1z', 7: '0x,0z', 8: '0x,1z', 9: '0x,0z', 10: '0x,0z', 11: '1x,1z', 12: '0x,0z', 13: '0x,1z', 14: '1x,0z', 15: '0x,0z'}
Bob send double matchings: [1, 2, 4, 7, 8, 12, 14]
Alice send usable frames: [(1, 2, 14), (1, 4, 14), (1, 8, 14), (2, 4, 14), (2, 8, 14), (4, 8, 14), (7, 8, 14)]
Bob send sifting string: ['010', '010', '010', '000', '000', '000', '011']
Bob shared key: 001001001001001001110
Alice shared key: 001001001001001001110


In [2]:
from error_partial_key_recovery import attack

# 011,000 no existe
# 011,110 no existe
# 110,000 no existe
# 110.001 no existe
# 110,111 no existe
# 111,010 no existe
# 111,100 no existe

DISAMBIGUATED_SS = [
    "011,001"
    "011,111",
    "110,110",
    "111,011",
    "111,101"
]


public, private = execute_qkd(64, DEBUG = True, depolarize_probability = 1)
usable_frames, sifting_string, measured_string = public
alice_key, bob_key = private

assert len(sifting_string) == len(measured_string)
# assert alice_key == bob_key

key_blocks = [ bob_key[i : i + 3] for i in range(0, len(alice_key), 3)  ]

key_recovered = attack(usable_frames, sifting_string, measured_string, DEBUG = False)
key_recovered_blocks = [ key_recovered[i : i + 3] for i in range(0, len(key_recovered), 3)  ]

# Compute SS
SS = [ f"{sifting_string[i]},{measured_string[i]}" for i in range(len(sifting_string)) ]

# Considering errors
error_key_blocks = []
error_key_recovered = []

for i in range(len(SS)):
    if SS[i] in DISAMBIGUATED_SS:
        error_key_blocks.append(key_blocks[i])
        error_key_recovered.append(key_recovered_blocks[i])
    else:
        continue

for x, y in zip(error_key_blocks, error_key_recovered):
    print(x, y)

# assert error_key_blocks == error_key_recovered
# print("All bits recovered!")

Key exchange failed, both keys are not equal. Please, try again.


NameError: name 'frames_101_111' is not defined

In [21]:
%%time

from partial_key_recovery import attack

VALID_SS = [
    "111,110",
    "111,011",
    "110,110",
    "110,011",
    "101,111",
    "101,100",
    "101,001",
    "100,111",
    "100,100",
    "100,010",
    "100,001",
    "011,111",
    "011,100",
    "011,010",
    "011,001",
    "010,111",
    "010,100",
    "010,010",
    "010,001",
    "001,110",
    "000,110",
    "000,011"
]

public, private = execute_qkd(256, DEBUG = False)
usable_frames, sifting_string, measured_string = public
alice_key, bob_key = private

assert alice_key == bob_key

key_recovered, rec = attack(usable_frames, sifting_string, measured_string, DEBUG = False)

private_key_blocks = [ alice_key[i : i + 3] for i in range(0, len(private[0]), 3)  ]
key_recovered_blocks = [ key_recovered[i : i + 3] for i in range(0, len(key_recovered), 3)  ]

SS = [ f"{sifting_string[i]},{measured_string[i]}" for i in range(len(sifting_string)) ]

key_blocks = []
key_recovered = []

SS_LEFT = []
for i in range(len(SS)):
    if SS[i] in VALID_SS:
        key_blocks.append(private_key_blocks[i])
        key_recovered.append(key_recovered_blocks[i])
        if key_recovered_blocks[i] == "   ":
            SS_LEFT.append(SS[i])
    else:
        continue

bits_recovered = 0
for x, y in zip(key_blocks, key_recovered):
    if x != y and y != "   ":
        print(x, y)
    elif x == y:
        bits_recovered += 3

bits_recovered / (len(key_blocks) * 3), set(SS_LEFT)

CPU times: user 1h 30min 22s, sys: 0 ns, total: 1h 30min 22s
Wall time: 1h 30min 12s


(1.0, set())

In [8]:
from partial_key_recovery import attack

public, private = execute_qkd(256, DEBUG = False)
usable_frames, sifting_string, measured_string = public
alice_key, bob_key = private

assert alice_key == bob_key

key_recovered, rec = attack(usable_frames, sifting_string, measured_string, DEBUG = False)

private_key_blocks = [ alice_key[i : i + 3] for i in range(0, len(private[0]), 3)  ]
key_recovered_blocks = [ key_recovered[i : i + 3] for i in range(0, len(key_recovered), 3)  ]

bits_recovered = 0
for x, y in zip(private_key_blocks, key_recovered_blocks):
    if x != y and y != "   ":
        print(x, y)
    elif x == y:
        bits_recovered += 3

bits_recovered / len(alice_key)

KeyboardInterrupt: 

In [5]:
from partial_key_recovery import attack

public, private = execute_qkd(256, DEBUG = False)
usable_frames, sifting_string, measured_string = public
alice_key, bob_key = private
assert alice_key == bob_key

key_recovered = attack(usable_frames, sifting_string, DEBUG = False)

private_key_blocks = [ alice_key[i : i + 2] for i in range(0, len(private[0]), 2)  ]
key_recovered_blocks = [ key_recovered[i : i + 2] for i in range(0, len(key_recovered), 2)  ]

for x, y in zip(private_key_blocks, key_recovered_blocks):
    print(x, y)

11   
11   
11   
01   
01   
11   
11   
01   
01   
11   
01   
11   
11   
01   
11   
11   
11   
11   
01   
01   
01   
01   
11   
01   
11   
01   
11   
01   
01   
01   
11   
11   
01   
01   
11   
11   
01   
11   
11   
01   
11   
01   
01   
01   
01   
01   
11   
11   
01   
11   
11   
01   
11   
01   
11   
11   
01   
00   
00   
10 10
10 10
00 00
00 00
10   
00 00
00 00
00   
10   
00 00
10   
00   
10   
00   
10   
10 10
10   
10   
00   
10 10
10 10
00   
00   
10   
10   
10   
10 10
00 00
10   
10 10
00 00
00   
00   
00   
00   
10   
00 00
10 10
00 00
00 00
00   
10   
10   
00 00
00   
10   
00   
10 10
10 10
10   
10 10
00 00
00   
10   
10   
10 10
10 10
10   
10   
10   
00   
10 10
10   
00   
00   
00   
10   
00   
10   
00   
10   
00   
10   
00   
10   
10   
10   
10   
00   
10   
00   
10   
00   
10   
10   
10   
10   
10   
10   
10   
00   
10   
00   
10   
00   
10   
10   
10   
00   
00   
00   
10   
10   
00   
10   
10   
10   
10  

In [133]:
def key_recovery_metrics(bits):
    public, private = execute_qkd(bits, DEBUG = False)
    usable_frames, sifting_string = public
    alice_key, bob_key = private
    assert alice_key == bob_key

    key_recovered = attack(usable_frames, sifting_string, DEBUG = False)

    private_key_blocks = [ alice_key[i : i + 2] for i in range(0, len(private[0]), 2)  ]
    key_recovered_blocks = [ key_recovered[i : i + 2] for i in range(0, len(key_recovered), 2)  ]

    bits_recovered = 0
    for x, y in zip(private_key_blocks, key_recovered_blocks):
        if y != "  ":
            bits_recovered += 2
    
    return bits_recovered / len(alice_key)

In [142]:
from numpy import mean

# mean_recovered_per = []
bits = 256

for _ in range(1000):
    recovered_per = key_recovery_metrics(bits)
    mean_recovered_per.append(recovered_per)
    
mean(mean_recovered_per)

0.41571024930173567